# ETL Project in Python and MySql



## Extraction from CSV files

In [3]:
#import required dependencies here:
import pandas as pd
import datetime
from sqlalchemy import create_engine, inspect
import os
from dotenv import load_dotenv
import pymysql
import config

In [11]:
#read csv datasets
df1 = pd.read_csv("./Resources/SupermartGrocerySales.csv")
df1.head()

,Order ID,Customer Name,Category,Sub Category,City,Order Date,Region,Sales,Discount,Profit,State
0,OD1,Harish,Oil & Masala,Masalas,Vellore,11-08-2017,North,1254,0.12,401.28,Tamil Nadu
1,OD2,Sudha,Beverages,Health Drinks,Krishnagiri,11-08-2017,South,749,0.18,149.80,Tamil Nadu
2,OD3,Hussain,Food Grains,Atta & Flour,Perambalur,06-12-2017,West,2360,0.21,165.20,Tamil Nadu
3,OD4,Jackson,Fruits & Veggies,Fresh Vegetables,Dharmapuri,10-11-2016,South,896,0.25,89.60,Tamil Nadu
4,OD5,Ridhesh,Food Grains,Organic Staples,Ooty,10-11-2016,South,2355,0.26,918.45,Tamil Nadu


# Transform

In [12]:
#drop last column since all data is of tamil nadu only

df1.drop(df1.columns[[10]], axis=1, inplace=True)
df1.head()

,Order ID,Customer Name,Category,Sub Category,City,Order Date,Region,Sales,Discount,Profit
0,OD1,Harish,Oil & Masala,Masalas,Vellore,11-08-2017,North,1254,0.12,401.28
1,OD2,Sudha,Beverages,Health Drinks,Krishnagiri,11-08-2017,South,749,0.18,149.80
2,OD3,Hussain,Food Grains,Atta & Flour,Perambalur,06-12-2017,West,2360,0.21,165.20
3,OD4,Jackson,Fruits & Veggies,Fresh Vegetables,Dharmapuri,10-11-2016,South,896,0.25,89.60
4,OD5,Ridhesh,Food Grains,Organic Staples,Ooty,10-11-2016,South,2355,0.26,918.45


In [18]:
# filtering out customers who generated high profits (Profits > 700) and sort the discount spend by ascending

df2 = df1.query('Profit>700')
df2 = df2.sort_values(by=['Discount'], ascending=True)
df2.head()

,Order ID,Customer Name,Category,Sub Category,City,Order Date,Region,Sales,Discount,Profit
436,OD437,Ram,"Eggs, Meat & Fish",Eggs,Dharmapuri,06-12-2017,Central,2200,0.1,792.00
489,OD490,Vidya,Snacks,Chocolates,Ramanadhapuram,06-09-2015,Central,2232,0.1,870.48
256,OD257,Ravi,Beverages,Soft Drinks,Ooty,11/28/2016,Central,2417,0.1,990.97
9685,OD9686,Komal,Snacks,Chocolates,Tenkasi,7/23/2018,Central,2122,0.1,721.48
4621,OD4622,Komal,Oil & Masala,Spices,Viluppuram,4/13/2016,West,2081,0.1,749.16


In [23]:
# Picking the 50 most lucrative customers by taking the most highest profits and lowest discounts used

df_top_50 = df2.sort_values(by=['Profit', 'Discount'], ascending=[False, True]).head(50)

# Display the top 50 customers
df_top_50.head()

,Order ID,Customer Name,Category,Sub Category,City,Order Date,Region,Sales,Discount,Profit
3159,OD3160,Haseena,Bakery,Cakes,Cumbum,09-04-2017,Central,2491,0.26,1120.95
3467,OD3468,Verma,Fruits & Veggies,Fresh Fruits,Theni,2/16/2015,Central,2490,0.24,1120.50
3436,OD3437,Yadav,Bakery,Breads & Buns,Theni,12/16/2015,Central,2469,0.29,1111.05
8134,OD8135,Aditi,Bakery,Biscuits,Coimbatore,9/18/2016,East,2452,0.18,1103.40
9782,OD9783,Komal,Snacks,Cookies,Karur,3/29/2017,Central,2450,0.21,1102.50


# Load

# Schema Table

```sql

CREATE DATABASE etl_project1;

CREATE TABLE IF NOT EXISTS etl_project1.customer_orders (
    `Order ID` INT PRIMARY KEY,
    `Customer Name` VARCHAR(100),
    `Category` VARCHAR(100),
    `Sub Category` VARCHAR(100),
    `City` VARCHAR(100),
    `Order Date` DATE,
    `Region` VARCHAR(50),
    `Sales` DECIMAL(10, 2),
    `Discount` DECIMAL(5, 2),
    `Profit` DECIMAL(10, 2)
);
```

In [ ]:
# Get MySQL credentials from environment variables
username = config.MYSQL_USER
password = config.MYSQL_PASSWORD
host = 'localhost'  
port = '3306' 
database = 'etl_project1' 

# Create SQLAlchemy engine for MySQL
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}', echo=False)


In [28]:
# Check if mysql connection is established

try:
    with engine.connect() as connection:
        print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")

Connection successful!


In [30]:
try:
    df_top_50.to_sql(name='loyal_customers',con=engine,index=False, if_exists='replace')
    print('Sucessfully written to Database!!!')

except Exception as e:
    print(e)

Sucessfully written to Database!!!


In [32]:
# Confirm data has been added to the table
pd.read_sql_query("SELECT * FROM loyal_customers", con=engine).head()

,Order ID,Customer Name,Category,Sub Category,City,Order Date,Region,Sales,Discount,Profit
0,OD3160,Haseena,Bakery,Cakes,Cumbum,09-04-2017,Central,2491,0.26,1120.95
1,OD3468,Verma,Fruits & Veggies,Fresh Fruits,Theni,2/16/2015,Central,2490,0.24,1120.50
2,OD3437,Yadav,Bakery,Breads & Buns,Theni,12/16/2015,Central,2469,0.29,1111.05
3,OD8135,Aditi,Bakery,Biscuits,Coimbatore,9/18/2016,East,2452,0.18,1103.40
4,OD9783,Komal,Snacks,Cookies,Karur,3/29/2017,Central,2450,0.21,1102.50
